In [21]:
from ngsolve import *
import netgen.gui
# import netgen.gui
from ngsolve import Draw, Redraw # just for visualization

In [22]:
import matplotlib.pyplot as plt
import numpy as np

In [23]:
from netgen.csg import *

left  = Plane (Pnt(0,0,0), Vec(-1,0,0) ).bc("dirichlet")
right = Plane (Pnt(1,1,1), Vec( 1,0,0) ).bc("dirichlet")
front = Plane (Pnt(0,0,0), Vec(0,-1,0) ).bc("dirichlet")
back  = Plane (Pnt(1,1,1), Vec(0, 1,0) ).bc("dirichlet")
bot   = Plane (Pnt(0,0,0), Vec(0,0,-1) ).bc("bottom")
top   = Plane (Pnt(1,1,1), Vec(0,0, 1) ).bc("top")


cube = left * right * front * back * bot * top

In [24]:
geo = CSGeometry()
geo.Add (cube)

mesh = geo.GenerateMesh(maxh=0.5)
#Redraw()
mesh = Mesh(mesh)
Draw(mesh)

 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Surface 1 / 6
 Optimize Surface
 Surface 2 / 6
 Optimize Surface
 Surface 3 / 6
 Optimize Surface
 Surface 4 / 6
 Optimize Surface
 Surface 5 / 6
 Optimize Surface
 Surface 6 / 6
 Optimize Surface
 Delaunay meshing
 start tetmeshing
 Success !
 Remove Illegal Elements
 Volume Optimization


In [25]:
# set order of method
orderP = 3
# adaptive parameter theta=1 corresponds to uniform ref.
theta = 1;
# set max dofs
maxDOFs = 250000
nDOFvisualize = maxDOFs
# Preconditioner
precNAME = "multigrid"
# precision
precCG = 1e-8

# filename to store data (mesh+solution) for visualization in e.g. paraview
fNAME = "ExampleX"

In [26]:
# Define data

#
f = sin(pi*x)*sin(pi*y)+2*pi*pi*z*z/2*sin(pi*x)*sin(pi*y)
g1 = CF(0)
g2 = CF(0)
v0 = CF(0)
sigma10 = CF(0)
sigma20 = CF(0)
vEx = z*sin(pi*x)*sin(pi*y)


#
#f = 0
#g1 = 0
#g2 = 0
#u0 = (1-2*Norm(x-0.5))*(1-2*Norm(y-0.5))
#v0 = u0.Diff(x)+u0.Diff(y)
#sigma10 = -v0.Diff(x)
#sigma20 = -v0.Diff(y)

# pulse
#x0 = 0.2
#y0 = 0.2
#v0 = -exp(-200*((x-x0)*(x-x0)+(y-y0)*(y-y0)))
#sigma10 = exp(-200*((x-x0)*(x-x0)+(y-y0)*(y-y0)))
#sigma20 = exp(-200*((x-x0)*(x-x0)+(y-y0)*(y-y0)))
#f = CF(0)
#g1 = 400*exp(-200*((x-x0-z)*(x-x0-z)+(y-y0-z)*(y-y0-z)))*(y-y0-z)
#g2 = 400*exp(-200*((x-x0-z)*(x-x0-z)+(y-y0-z)*(y-y0-z)))*(x-x0-z)


In [27]:
# FEM Spaces
V = H1(mesh, order=orderP, dirichlet="dirichlet")
SIGMA = H1(mesh, order=orderP)

fesm = V*SIGMA*SIGMA

In [28]:
v,sigma1,sigma2 = fesm.TrialFunction()
w,tau1,tau2 = fesm.TestFunction()

In [29]:
# Define and assemble bilinear form
blf = BilinearForm(fesm)
blf+= ( (grad(v)[2]-grad(sigma1)[0]-grad(sigma2)[1])*(grad(w)[2]-grad(tau1)[0]-grad(tau2)[1])  )*dx
blf+= ( (grad(sigma1)[2]-grad(v)[0])*(grad(tau1)[2]-grad(w)[0])  )*dx
blf+= ( (grad(sigma2)[2]-grad(v)[1])*(grad(tau2)[2]-grad(w)[1])  )*dx

# add boundary terms
blf+= ( v*w + sigma1*tau1 + sigma2*tau2 )*ds("bottom")


# register preconditioner
prec = Preconditioner(blf, precNAME)


blf.Assemble()

In [30]:
# Define Linear form
rhs = LinearForm(fesm)
rhs+= (f*(grad(w)[2]-grad(tau1)[0]-grad(tau2)[1]) + g1*(grad(tau1)[2]-grad(w)[0])+ g2*(grad(tau2)[2]-grad(w)[1]))*dx
rhs+= ( v0*w + sigma10*tau1 + sigma20*tau2 )*ds("bottom")

rhs.Assemble()


In [31]:
# grid function
gfm = GridFunction(fesm)
gfv, gfsigma1, gfsigma2 = gfm.components

In [32]:
#Define boundary2elements mapping
def GetB2E():
    #start = time.time()
   
    f2eAUX = np.zeros(mesh.nface,dtype=int)
    
    for el in mesh.Elements(VOL):
        f2eAUX[el.faces[0].nr] = el.nr
        f2eAUX[el.faces[1].nr] = el.nr
        f2eAUX[el.faces[2].nr] = el.nr
        f2eAUX[el.faces[3].nr] = el.nr
    
    nrBND = 0;
    for el in mesh.Elements(BND):
        nrBND += 1
    
    b2e = np.zeros(nrBND,dtype=int)
    
    
    for el in mesh.Elements(BND):
        b2e[el.nr] = f2eAUX[el.faces[0].nr]
    
    
    #end = time.time()
    #print("    Computing took ", (end - start), " seconds")
    return b2e

In [33]:
# realization of doerfler marking
def MarkBulk(eta,theta):
    indicators_sorted=np.sort(eta.NumPy())
    indicators_sorted[::-1] = indicators_sorted
    idx=np.argsort(eta.NumPy())
    idx[::-1]=idx

    

    sumind = np.cumsum(indicators_sorted)

    
    tmp = sumind>=sumind[len(sumind)-1]*theta
    jdx = np.min(np.where(tmp==True))

    
    
    marked = idx[0:jdx+1]
    #print(np.arange(0,jdx+1))
    #print(marked)

    tmp = np.full(len(eta.NumPy()), False)
    tmp[marked] = True
    #print(tmp)
    return tmp

In [34]:
l = []    # l = list of estimated total error
errExact = [] # if exact solution is known

def CalcError():

    # compute estimator:
    gfm.Update()
    gfv.Update()
    gfsigma1.Update()
    gfsigma2.Update()

    errVol = (grad(gfv)[2]-grad(gfsigma1)[0]-grad(gfsigma2)[1]-f)*(grad(gfv)[2]-grad(gfsigma1)[0]-grad(gfsigma2)[1]-f)
    errVol += (grad(gfsigma1)[2]-grad(gfv)[0]-g1)*(grad(gfsigma1)[2]-grad(gfv)[0]-g1)
    errVol += (grad(gfsigma2)[2]-grad(gfv)[1]-g2)*(grad(gfsigma2)[2]-grad(gfv)[1]-g2)
    eta2 = Integrate(errVol, mesh, VOL, element_wise=True)
    
    # boundary errors
    errBou2 = Integrate((gfv-v0)*(gfv-v0),mesh,BND,element_wise=True,definedon=mesh.Boundaries("bottom"))
    errBou2+= Integrate((gfsigma1-sigma10)*(gfsigma1-sigma10),mesh,BND,element_wise=True,definedon=mesh.Boundaries("bottom"))
    errBou2+= Integrate((gfsigma2-sigma20)*(gfsigma2-sigma20),mesh,BND,element_wise=True,definedon=mesh.Boundaries("bottom"))

    
    tmp = eta2.NumPy()
    b2e = GetB2E() # compute boundary to volume element relation
    tmp[b2e] = tmp[b2e] + errBou2 # update error estimator (this adds directly to eta2)

    if(theta<1):
        mesh.ngmesh.Elements3D().NumPy()["refine"] = MarkBulk(eta2,theta)

    l.append ((fesm.ndof, sqrt(sum(eta2))))
    print("ndof =", fesm.ndof, " ErrEst =", sqrt(sum(eta2)))

    errTMP = Integrate( (vEx-gfv)*(vEx-gfv), mesh, VOL,element_wise=True)
    errExact.append( (fesm.ndof,sqrt(sum(errTMP))))

In [ ]:
level = 0
steps = []
while fesm.ndof < maxDOFs:
    if level > 0:
        mesh.Refine()
    fesm.Update()
    gfm.Update()


    with TaskManager():
        blf.Assemble()
        rhs.Assemble()

        # Conjugate gradient solver
        inv = CGSolver(blf.mat, prec.mat, maxsteps=1000, precision=precCG)

        gfm.vec.data = inv * rhs.vec

    steps.append ( (fesm.ndof, inv.GetSteps()) )
    
    level = level+1
    CalcError() # compute error and mark elements

    if(fesm.ndof<nDOFvisualize):
        Draw(gfv)
        Redraw (blocking=True)


0 2.57563
1 0.126331
2 0.0655272
3 0.0291994
4 0.00873755
5 0.00472099
6 0.00259417
7 0.00150696
8 0.000535435
9 0.000284036
10 0.000165913
11 8.63873e-05
12 4.46799e-05
13 2.298e-05
14 1.18587e-05
15 5.47666e-06
16 2.67317e-06
17 1.41554e-06
18 5.85378e-07
19 3.11842e-07
20 1.6964e-07
21 7.85559e-08
22 4.1061e-08
23 1.91687e-08
ndof = 1164  ErrEst = 0.06401643701546228
 Mesh bisection
 Bisection done
0 2.49202
1 0.275608
2 0.0685673
3 0.0311963
4 0.0214969
5 0.0161876
6 0.00996816
7 0.00698236
8 0.00566512
9 0.00464428
10 0.00326979
11 0.00209947
12 0.00149146
13 0.00120214
14 0.000958771
15 0.000692801
16 0.000461853
17 0.00030083
18 0.000220054
19 0.000154376
20 0.000116455
21 9.24983e-05
22 5.65547e-05
23 3.8817e-05
24 3.06238e-05
25 2.4112e-05
26 1.84279e-05
27 1.35479e-05
28 9.37415e-06
29 6.88017e-06
30 4.9374e-06
31 3.61384e-06
32 2.57599e-06
33 1.78676e-06
34 1.45244e-06
35 1.08012e-06
36 7.91509e-07
37 5.9487e-07
38 4.53275e-07
39 3.18255e-07
40 2.40313e-07
41 1.76546e-07
42 

In [ ]:
plt.yscale('log')
plt.xscale('log')
plt.xlabel("ndof")
plt.ylabel("estimator")
ndof,err = zip(*l)
hp = np.power((np.divide(1,ndof)*ndof[-1]),orderP/3)
plt.plot(ndof,err, "-*",ndof,hp*err[-1])

plt.ion()
plt.show()

In [ ]:
plt.yscale('log')
plt.xscale('log')
plt.xlabel("ndof")
plt.ylabel("estimator")
ndof,err = zip(*errExact)
hp = np.power((np.divide(1,ndof)*ndof[-1]),(orderP)/3)
plt.plot(ndof,err, "-*",ndof,hp*err[-1])

plt.ion()
plt.show()

In [ ]:
gfm.Update()
gfv.Update()
Draw(gfv)

In [ ]:
# VTKOutput object
fNAME = fNAME + "_ndof_" + str(fesm.ndof)
print("Saving into Filename " + fNAME + ".vtu")


vtk = VTKOutput(ma=mesh,
                coefs=[gfv,gfsigma1,gfsigma2],
                names = ["v","sigma1","gfsigma2"],
                filename=fNAME,
                subdivision=orderP-1)
# Exporting the results:
vtk.Do()
